# Librerias

In [1]:
import numpy as np
from numpy.random import rand,seed
from matplotlib import pyplot as plt
from time import time
import pandas as pd
from torchvision.datasets import MNIST

# Obteniendo el dataset

In [2]:
df_trainX, df_trainY = list(zip(*MNIST(root="",
                                       train=True,
                                       download=True,
                                       transform=np.asarray)))

df_testX, df_testY = list(zip(*MNIST(root="",
                                       train=False,
                                       download=True,
                                       transform=np.asarray)))

60.5%

100.0%


Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw



100.0%

Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw




100.0%

Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw




100.0%

Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw


# Función de Entrenamiento

In [ ]:
def train(n0, n1, v_fun, dv_fun, fun, grad, backtracking_fun,
          trainy, trainx, testy, testx,
          tam=None, noise=0.001, lam=0.001, toler=1e-3, beta=0.2,
          gam=2, gradient=False, dibuja_ind=False):

    # Tamao dataset
    if tam is None:
        select = np.where((trainy == n0) + (trainy == n1))[0]
        tam = select.shape[0]
    else:
        select = np.where((trainy == n0) + (trainy == n1))[0][:tam]
        
    x_train = trainx[select].reshape(tam, 28 ** 2)  # /255.
    x_train = (x_train - x_train.mean().astype(np.float32)) / x_train.std().astype(
        np.float32)  # Image preprocessing: we standarized by substracting the mean and dividing by the standard deviation
    select_test = np.where((testy == n0) + (testy == n1))[0]
    x_test = testx[select_test].reshape(select_test.shape[0], 28 ** 2)  # /255.
    x_test = (x_test - x_test.mean().astype(np.float32)) / x_test.std().astype(np.float32)

    if noise is not None:
        ruido = np.random.randn(*x_train.shape) * noise
        smallest_sv = np.linalg.svd(x_train + ruido)[1].min()
        if smallest_sv < 1e-6:
            ruido = np.random.randn(*x_train.shape) * noise
            smallest_sv = np.linalg.svd(x_train + ruido)[1].min()
        x_train += ruido
        x_test += np.random.randn(*x_test.shape) * noise
        print('smallest singular value =', smallest_sv)
    else:
        smallest_sv = np.linalg.svd(x_train)[1].min()
    x_train = np.concatenate([np.ones([x_train.shape[0], 1]), x_train], axis=1)
    y_train = trainy[select] == n1
    x_test = np.concatenate([np.ones([x_test.shape[0], 1]), x_test], axis=1)
    y_test = testy[select_test] == n1

    m = y_train.shape[0]
    print('Numbers:', n0, 'and', n1)
    print('m =', m)
    print('noise =', noise)
    print('lambda = ', lam)

    y = 2 * y_train - 1
    C = np.expand_dims(y, 1) * x_train.copy()
    n = x_train.shape[1]

    # cTx=(-4/3*lam+(16/9*lam**2+4*nc**4)**.5)/(2*nc**2)
    # sol=3/4/lam*c*(1-cTx**2)

    # def S2V(t, s):
    #     if t < -1 or t > 1 or s == 0:
    #         return 0
    #     else:
    #         return (3/2) * t * s

    x0 = 2 * (rand(n) - .5)

    x0 = 2 * (rand(n) - .5)
    sol_alg1 = alg1(x0, sigma=0.2, t0=1, c=C, m=m, n=n, lam=lam, beta=beta, gam=gam, tol=toler)
    print('Alg1 done! [n,m] = ', [n, m])
    print('f(x0)= ', fun(x0, C, m))

    print('Alg1 :', sol_alg1[1:])  # ,norm(sol-sol_alg1[0]))

    y_train0 = (y_train == 0)
    y_test0 = (y_test == 0)

    aciertos_train = ((v_fun(x_train @ sol_alg1[0]) >= 0.5) == y_train0).sum()
    aciertos_test = ((v_fun(x_test @ sol_alg1[0]) >= 0.5) == y_test0).sum()

    print(
        f'Aciertos en el conjunto de entrenamiento: {aciertos_train} de {y_train.shape[0]} ({(aciertos_train / y_train.shape[0]):2.2%})')
    print(
        f'Aciertos en el conjunto de test: {aciertos_test} de {y_test.shape[0]} ({(aciertos_test / y_test.shape[0]):2.2%})')

    if dibuja_ind:
        ind = np.where((v_fun(x_test @ sol_alg1[0]) >= 0.5) != y_test0)[0]
        num_row, num_col = 2, int(np.ceil(ind.shape[0] / 2))
        fig, axes = plt.subplots(num_row, num_col, figsize=(1.5 * num_col / 2, 2 * num_row / 2))
        for i in range(ind.shape[0]):
            ax = axes[i // num_col, i % num_col]
            ax.imshow(x_test[ind[i]][1:].reshape(28, 28), cmap='gray_r')
            ax.axis(False)
            ax.set_title('Label: {} '.format(testy[select_test][ind[i]]), fontsize=10)
        plt.tight_layout()
        plt.savefig('Misclassified_' + str(n0) + '_' + str(n1) + '_' + str(tam) + '_' + str(noise) + '.pdf',
                    bbox_inches='tight')
        plt.show()

    if gradient:
        sol_grad = grad(x0, tol=toler)
        print('Grad done! [n,m] = ', [n, m])
        print('Grad :', sol_grad[1:])  # ,norm(sol-sol_grad[0]))

        aciertos_grad_train = ((v_fun(x_train @ sol_grad[0]) >= 0.5) == y_train0).sum()
        aciertos_grad_test = ((v_fun(x_test @ sol_grad[0]) >= 0.5) == y_test0).sum()
        print(
            f'Aciertos en el conjunto de entrenamiento: {aciertos_grad_train} de {y_train.shape[0]} ({(aciertos_grad_train / y_train.shape[0]):2.2%})')
        print(
            f'Aciertos en el conjunto de test: {aciertos_grad_test} de {y_test.shape[0]} ({(aciertos_grad_test / y_test.shape[0]):2.2%})')
        return smallest_sv, sol_alg1, y_train.shape[0], aciertos_train, y_test.shape[
            0], aciertos_test, sol_grad, aciertos_grad_train, aciertos_grad_test
    else:
        return smallest_sv, sol_alg1, y_train.shape[0], aciertos_train, y_test.shape[0], aciertos_test